--- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device

Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k

The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

    cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
        cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
        cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
        cd / switches the current directory to the outermost directory, /.
    ls means list. It prints out all of the files and directories immediately contained by the current directory:
        123 abc means that the current directory contains a file named abc with size 123.
        dir xyz means that the current directory contains a directory named xyz.

Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)

Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

    The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
    The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
    Directory d has total size 24933642.
    As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?


In [2]:
def open_file():
    """open the input file"""
    with open('chris_davis_input_day7.txt') as f:
        input_data = f.readlines()
    return input_data

In [207]:
def get_commands_and_outputs(input_data):
    """Gets ordered commands and respective outputs as a dictionary"""
    command_counter = -1
    commands = {}
    outputs = {}
    for row in input_data:
        stripped_row = row.strip()
        if stripped_row[0] == '$':
            command_counter += 1
            commands[command_counter] = stripped_row[2:]
        else:
            if command_counter not in outputs:
                outputs[command_counter] = []
            outputs[command_counter].append(stripped_row)
    return commands, outputs

In [212]:
class directory():
    def __init__(self, name = None, parent = None, children = [], sum_file_contents = 0):
        self.name = name
        self.parent = parent
        self.children = children
        self.sum_file_contents = sum_file_contents
    
    def get_full_size(self):
        full_size = self.sum_file_contents
        for child in self.children:
            full_size += child.get_full_size()
        return full_size

    def add_child(self, child):
        if child not in self.children:
            self.children.append(child)
            
    def get_full_path(self):
        if self.parent:
            return f'{self.parent.get_full_path()} -> {self.name}' 
        else:
            return self.name

In [213]:
def transit_through_directory(commands):
    """
    Transits through the directories, following the commands.
    Returns a dictionary with a key for each command and value for the directory we are currently in.
    
    Also creates directories of the directory class, which determines the filestructure
    """
    locations = {} # {step: directory_name}
    complete_locations = {} # {step: full_path_to_directory}
    directory_dict = {} # {full_path_to_directory: directory}
    level = 0
    for i in range(0, max(commands.keys()) + 1):
        print(f'step {i}, following command {commands[i]}')
        if commands[i] == 'ls':
            print('\tprinting ls')
            # stay at same location
            locations[i] = locations[i - 1]
            complete_locations[i] = complete_locations[i-1]
        elif commands[i] == 'cd /':
            level = 0
            print(f'\tmoved to base directory. Now at level {level}')
            locations[i] = '/'
            complete_locations[i] = '/'
            
            if '/' not in directory_dict.keys():
                print('\tcreated base directory')
                directory_dict['/'] = directory('/', None, [], 0)
                
        elif commands[i] == 'cd ..':
            level = level - 1
            print(f'\tmoved back to parent directory {directory_dict[complete_locations[i-1]].parent.get_full_path()}. Now at level {level}')
            locations[i] = directory_dict[complete_locations[i-1]].parent.name
            complete_locations[i] = directory_dict[complete_locations[i-1]].parent.get_full_path()

        else:
            level = level + 1
            print(f'\tmoved to directory {commands[i][3:]}, creating new directory with parent {directory_dict[complete_locations[i-1]].get_full_path()} and adding child to {directory_dict[complete_locations[i-1]].get_full_path()}. Now at level {level}')
            locations[i] = commands[i][3:]
            complete_locations[i] = f'{directory_dict[complete_locations[i-1]].get_full_path()} -> {locations[i]}'
            if complete_locations[i] not in directory_dict.keys():
                directory_dict[complete_locations[i]] = directory(locations[i], directory_dict[complete_locations[i-1]], [], 0) 
                directory_dict[complete_locations[i-1]].add_child(directory_dict[complete_locations[i]])
    return locations, complete_locations, directory_dict

In [214]:
def get_contents(outputs):
    """
    Determine what is contained in each directory, based on commands.
    Splits contents into a dictionary of summed space from files and a list of directories
    """
    contents = {}
    for step, dir_output in outputs.items():
        contained_directories = []
        summed_space = 0
        for item in dir_output:
            if item[:3] == 'dir':
                contained_directories.append(item[4:])
            else:
                summed_space += int(item.split()[0])
            
        contents[step] = {'summed_space': summed_space, 'contained_directories': contained_directories}
    return contents

In [215]:
input_data = open_file()

In [216]:
commands, outputs = get_commands_and_outputs(input_data)

In [217]:
locations, complete_locations, directory_dict = transit_through_directory(commands)

step 0, following command cd /
	moved to base directory. Now at level 0
	created base directory
step 1, following command ls
	printing ls
step 2, following command cd bntdgzs
	moved to directory bntdgzs, creating new directory with parent / and adding child to /. Now at level 1
step 3, following command ls
	printing ls
step 4, following command cd ..
	moved back to parent directory /. Now at level 0
step 5, following command cd hsswswtq
	moved to directory hsswswtq, creating new directory with parent / and adding child to /. Now at level 1
step 6, following command ls
	printing ls
step 7, following command cd bsjbvff
	moved to directory bsjbvff, creating new directory with parent / -> hsswswtq and adding child to / -> hsswswtq. Now at level 2
step 8, following command ls
	printing ls
step 9, following command cd dmnt
	moved to directory dmnt, creating new directory with parent / -> hsswswtq -> bsjbvff and adding child to / -> hsswswtq -> bsjbvff. Now at level 3
step 10, following comma

In [218]:
contents = get_contents(outputs)

In [222]:
for step in outputs.keys():
    directory_dict[complete_locations[step]].sum_file_contents = contents[step]['summed_space']

In [241]:
sum_sizes = 0
for directory_path, directory in directory_dict.items():
    size = directory.get_full_size()
    if size <= 100000:
        sum_sizes += size

In [240]:
sum_sizes

1206825

--- Part Two ---

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:

    Delete directory e, which would increase unused space by 584.
    Delete directory a, which would increase unused space by 94853.
    Delete directory d, which would increase unused space by 24933642.
    Delete directory /, which would increase unused space by 48381165.

Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?


In [243]:
directory_dict['/'].get_full_size()

48729145

In [244]:
available_space = 70000000 - directory_dict['/'].get_full_size()

In [245]:
available_space

21270855

In [246]:
needed_space = 30000000 - available_space

In [247]:
needed_space

8729145

In [248]:
best_directory = None
best_space = 0
for directory_path, directory in directory_dict.items():
    if directory.get_full_size() > needed_space:
        if not best_directory:
            best_directory = directory
            best_space = directory.get_full_size()
        elif best_space > directory.get_full_size():
            best_directory = directory
            best_space = directory.get_full_size()
best_space

9608311